### Generics

Bəzən biz funksiya və yeni mürəkkəb tip quranda onun istifadə etdiyi parametrlərin və   
dəyişənlərin tipini istifadə zamanı müəyyən edilməsini istəyirik.  
Aydın məsələdir ki bu boş interfeysdən fərqli olaraq daha sürətli olacaq.  
Buna Go dilində bir çox dillərdə olduğu kimi generics type deyilən yanaşma ilə nail olunur.  
Go dilinin yeni versiyalarında (>=1.18) artıq bunu istifadə etmək mümkündür. 

Tip elan edilməsinə yenidən baxaq:  
<pre>

 
type ad [TipParametri] tip 

Burada TipParametri:
[ ] arasında ad və Məhdudiyyət cütlüklərindən ibarət olan siyahı olur
Məsələn [T Constraint], [T Məhdudiyyət1, V Məhdudiyyət2, ...]

(Tip Məhdudiyyətlərini aşağıda xüsusi bölmədə izah edəcəyik.  )

Yeni tipimizə metod vermək üçün  

func (x * ad[T]) metod_adı(parametrlər... ) nəticə... { ...}

</pre>

Həmçinin generic funksiya elan edilməsinə baxaq:
<pre>

func G[T Constraint](p T) { ... }

</pre>

Hələki tip məhdudiyyəti kimi **any** götürək hansı ki bütün tipləri götürür.  
Onda Generics-ə nümunə olaraq bunları göstərə bilərik  
```Go
//generic List
type List[T any] struct {
	next  *List[T]
	value T
}

// List[T] biz üçün metod vermə qaydasına nümunə
func (l *List[T]) Len() int  { ... }

//generic Print funksiyamız
func Print[T any](s []T) {
    for _, v := range s {
        fmt.Println(v)
    }
}

```

In [4]:

func Print[T any](s []T) {
    for _, v := range s {
        fmt.Println(v)
    }
}

%%
Print[int]([]int{1, 2, 3})
Print[string]([]string{"alma","banan", "kivi"})

1
2
3
alma
banan
kivi


Yuxarıdakı çağırış tərzini sadələşdirmək də olar.  
Go bizim yerimizə parametrdən tipi təyin edəcək.  


In [20]:
%%
Print([]string{"alma","banan", "kivi", "armud"})

alma
banan
kivi
armud


#### Type Contraints  (Tip məhdudiyyətləri)  

Tip məhdudiyyəti müvafiq tip parametri üçün icazə verilən tip arqumentlər dəstini təyin edir.  
Bu dəstəklənən əməliyyatlara nəzarət edən interfeysdir.  

Tərifdən də aydın olur ki biz niyə any istifadə etmişik. Çünki any boş interfeysdir.  

Aşağıdakı tip məhdudiyyətlərinin elanına fikir verək  
<pre>
[T any]  //[T interface{}]
[T interface{[]P}]
[T interface{~int}]
[T interface{int|string}]
</pre>

Fikir verdinizsə digər interfeys yazıları bizə tanış deyil.  
Biz interfeys daxilində ya boşluq ya da funksiya (func-suz) elanı istifadə edirdik.  
Bu tip interfeys sadə interfeysdən fərqlidir və buna ümumi interfeys deyirik. (General İnterface)

##### General Interface (Ümumi interfeys)  
Ümumi interfeysin elanı.  
Asan olsun deyə qaydalar şəklində yox göstərməklə təyin nümunlərinə baxaq:  
```Go
// Interfeys yalnız "tip" tipini əhatə edir.
interface {
	tip
}
// Məsələn:
interface {
	int
}

// Interfeys alt tipi "tip" olan bütün tipləri əhatə edir.
// yəni alt tipi "tip" olan bütün tiplər interfeysin tip çoxluğunda olur.
interface {
	~tip
}

// Məsələn:
interface {
	~int
}
// MyInt alt tipi int olduğundan interfeys tərəfindən qəbul olunacaq
type MyInt int 


// Interfeys alt tipi "tip" olan və həmçinin "metod_adı" metodu elan edilmiş tipləri əhatə edir.  
// yəni interfeysin tip çoxluğunda olur.
interface {
	~tip
	metod_adı(parameterlər) nəticələr
}

// Məsələn:
interface {
	~int
	String() string
}
type MyInt2 int

func (v MyInt2) String() string {
    ...
}

// Aşağıdakı misalda isə heç bir tip əhatə olunmayacaq. Çünki bir tipin həm int, həm string tipi olması mümkün deyil.
interface {
	int
	string
}

//Qeyd edək ki | məntiqi və ya bağlayıcısı ilə də məhdudiyyət interfeysi hazırlaya bilərik

// Interfeys "tip1" və ya "tip2" və ya "tip3" tipini əhatə edir.
interface {
	tip1 | tip2 | tip3
}

// Məsələn:

//alt tipi int və ya string olan tiplər
interface {
	~int | string
}


// https://pkg.go.dev/golang.org/x/exp/constraints 
type Signed interface {
	~int | ~int8 | ~int16 | ~int32 | ~int64
}

type Unsigned interface {
	~uint | ~uint8 | ~uint16 | ~uint32 | ~uint64 | ~uintptr
}

type Integer interface {
	Signed | Unsigned
}

```


Daha dəqiq və ətraflı qaydalar və istifadə toplumu üçün dilə baxa bilərsiniz.  
https://go.dev/ref/spec#Interface_types

Qeyd edək ki, bəzi tip məhdudiyyətlərini sadə üsulla da elan edə bilərik.   
Məsələn:  
<pre>
[T []P]                      // = [T interface{[]P}]
[T int]                     // = [T interface{int}]
[T ~int]                     // = [T interface{~int}]
[T int|string]               // = [T interface{int|string}]
</pre>

(Qeyd: bu halda [T int] -də int-i tip yox, tip məhdudiyyəti kimi başa düşmək daha düzgündür) 

Amma  nəzərə almaq lazımdır ki bu sadələşdirmə 
qarışıqlığa səbəb olsa onda onu xüsusi üsulla qeyd etmək lazımdır.  
Məsələn aşağıdakı C məhdudiyyəti üçün qarışıqlıq yarana bilər.  

<pre>
type X1[P *C] ..
type X2[P (C)] ...
type X3[P *C|Q] ...
</pre>

Bu qarşılıqlığı aradan qaldırmaq üçün ya təzədən məhdudiyyətləri interface{} almaq, ya da "," işarəsini sonuna əlavə etmək lazımdır  
<pre>
type X1[P interface{*C}] ...
type X1[P *C,] ...
</pre>

Dediyimiz qarşılıqlığa aid real nümunə verək və çıxan səhvə baxaq:

In [24]:
type Lx [ T *int] struct {
	val T
}

# gonb_93df4279 

 

 Cell[24]: Line 1 
 ./main.go:11:11: undefined: T
 
type List[T any] struct {
 head, tail *element[T]
}
 type Lx [ T *int] struct {
 val T
}

 

 

 Cell[24]: Line 1 
 ./main.go:11:14: int (type) is not an expression
 
type List[T any] struct {
 head, tail *element[T]
}
 type Lx [ T *int] struct {
 val T
}

 

 

 Cell[24]: Line 2 
 ./main.go:12:6: undefined: T
 
 head, tail *element[T]
}
type Lx [ T *int] struct {
 val T
 }
type element[T any] struct {

ERROR: failed to run "/usr/local/go/bin/go build -o /tmp/gonb_93df4279/gonb_93df4279": exit status 1

Gördüyünüz kimi Go bunu düzgün parse edə bilmədi. Bu adətən o zaman baş verirki  
yazılış ifadəyə oxşayır. İndi isə həmin problemi aradan qaldıran yazılışa baxaq:

In [6]:
// Normal interface{ } içində elanla
type Lx [ T interface{*int}] struct {
	val T
}

In [3]:
// Əlavə vergül yazmaqla
type Lx [ T *int,] struct {
	val T
}

##### comparable
Go dilində any interfeysi ilə yanaşı comparable interfeys məhdudiyyəti var.  
comparable məhdudiyyətinin əhatə etdiyi tiplər üzərində  müqayisə əməliyyatlarını ( ==, != ) həyata keçirmək mümkündür.  
Əks halda həmin tip comparable interfeys çoxluğuna daxil olmur. 
(Go 1.20-dən any də comparable tipin interfeys çoxluğundadır)  

In [21]:
func beraber[T comparable](x, y T) bool {
    if x == y {
        return true
    }
    return false
} 

%%
var a,b int
a=5
b=8
fmt.Println(beraber[int](a,b))
fmt.Println(beraber(b, b))


false
true


Məsələn bilirik ki funksiya tipləri müqayisə olunmur. 

In [32]:
 
%%
a:= func() int {return 5}
b:= func() int {return 8}
fmt.Println(beraber(a,b))


# gonb_93df4279 

 

 Cell[32]: Line 5 
 ./main.go:56:21: func() int does not satisfy comparable
 
	flag.Parse()
	a:= func() int {return 5}
	b:= func() int {return 8}
 fmt.Println(beraber(a,b))

ERROR: failed to run "/usr/local/go/bin/go build -o /tmp/gonb_93df4279/gonb_93df4279": exit status 1

Təəssüf ki, Go dilində öz tipimizi comparable çoxluğuna əlavə etmək üçün elan  
etdiyimiz tip üçün xüsusi == != metodlarını əlavə edə bilmirik.  Bu xüsusi qaydalara əsasən təyin edilir.  
Məsələn struct tipimizin bütün eleməntləri müqayisə edilən olduqda bizim struct da comparable interfeys çoxluğuna daxil olur.  
Tam qaydaları   
https://go.dev/ref/spec#Comparison_operators bölməsində tapmaq olar


Generics-ə aid List nümunəsinə baxaq.  
Bu nümunə https://gobyexample.com/generics götürülüb

In [22]:
type List[T any] struct {
    head, tail *element[T]
}
type element[T any] struct {
    next *element[T]
    val  T
} 
func (lst *List[T]) Push(v T) {
    if lst.tail == nil {
        lst.head = &element[T]{val: v}
        lst.tail = lst.head
    } else {
        lst.tail.next = &element[T]{val: v}
        lst.tail = lst.tail.next
    }
}
func (lst *List[T]) GetAll() []T {
    var elems []T
    for e := lst.head; e != nil; e = e.next {
        elems = append(elems, e.val)
    }
    return elems
}

%%
lst := List[int]{}
lst.Push(10)
lst.Push(13)
lst.Push(23)
fmt.Println("list:", lst.GetAll())

list: [10 13 23]
